In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler

In [3]:
column_names = ['Names', 'MCG','GVH' ,'LIP','CHG','AAC','ALM1','ALM2','SITE']
ecoli_df = pd.read_csv('./uci_repos/ecoli/ecoli.data',
                      sep="   |  ",
                                   names=column_names, 
                                   header=None, 
                                   engine="python")
ecoli_df

,Names,MCG,GVH,LIP,CHG,AAC,ALM1,ALM2,SITE
0,AAT_ECOLI,0.49,0.29,0.48,0.5,0.56,0.24,0.35,cp
1,ACEA_ECOLI,0.07,0.40,0.48,0.5,0.54,0.35,0.44,cp
2,ACEK_ECOLI,0.56,0.40,0.48,0.5,0.49,0.37,0.46,cp
3,ACKA_ECOLI,0.59,0.49,0.48,0.5,0.52,0.45,0.36,cp
4,ADI_ECOLI,0.23,0.32,0.48,0.5,0.55,0.25,0.35,cp
...,...,...,...,...,...,...,...,...,...
331,TREA_ECOLI,0.74,0.56,0.48,0.5,0.47,0.68,0.30,pp
332,UGPB_ECOLI,0.71,0.57,0.48,0.5,0.48,0.35,0.32,pp
333,USHA_ECOLI,0.61,0.60,0.48,0.5,0.44,0.39,0.38,pp
334,XYLF_ECOLI,0.59,0.61,0.48,0.5,0.42,0.42,0.37,pp


In [4]:
ecoli_df.describe()

,MCG,GVH,LIP,CHG,AAC,ALM1,ALM2
count,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000
mean,0.500060,0.500000,0.495476,0.501488,0.500030,0.500179,0.499732
std,0.194634,0.148157,0.088495,0.027277,0.122376,0.215751,0.209411
min,0.000000,0.160000,0.480000,0.500000,0.000000,0.030000,0.000000
25%,0.340000,0.400000,0.480000,0.500000,0.420000,0.330000,0.350000
50%,0.500000,0.470000,0.480000,0.500000,0.495000,0.455000,0.430000
75%,0.662500,0.570000,0.480000,0.500000,0.570000,0.710000,0.710000
max,0.890000,1.000000,1.000000,1.000000,0.880000,1.000000,0.990000


In [5]:
del ecoli_df['Names']
class_names = ecoli_df['SITE'].unique()
class_names

array(['cp', 'im', 'imS', 'imL', 'imU', 'om', 'omL', 'pp'], dtype=object)

In [6]:
ecoli_df.head()

,MCG,GVH,LIP,CHG,AAC,ALM1,ALM2,SITE
0,0.49,0.29,0.48,0.5,0.56,0.24,0.35,cp
1,0.07,0.40,0.48,0.5,0.54,0.35,0.44,cp
2,0.56,0.40,0.48,0.5,0.49,0.37,0.46,cp
3,0.59,0.49,0.48,0.5,0.52,0.45,0.36,cp
4,0.23,0.32,0.48,0.5,0.55,0.25,0.35,cp


In [7]:
minority_group = ['imL', 'omL', 'imS']
new_ecoli_df = ecoli_df[~ecoli_df['SITE'].isin(minority_group)]

In [8]:
labels = new_ecoli_df.iloc[:,-1:].copy().to_numpy() 
data = new_ecoli_df.iloc[:, 0:-1].copy().to_numpy()

# VAE Model

In [9]:
import tensorflow.compat.v1  as tf
tf.disable_v2_behavior()
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import os
import numpy as np

mb_size = 16
z_dim = 3
X_dim = 7
y_dim = 5
h_dim = 3
lr = 1e-3

    
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random.normal(shape=size, stddev=xavier_stddev)

X = tf.keras.Input(shape=(X_dim,))
c = tf.keras.Input(shape=(y_dim,))
z = tf.keras.Input(shape=(z_dim,))

Q_W1 = tf.Variable(xavier_init([X_dim + y_dim, h_dim]))
Q_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

Q_W2_mu = tf.Variable(xavier_init([h_dim, z_dim]))
Q_b2_mu = tf.Variable(tf.zeros(shape=[z_dim]))

Q_W2_sigma = tf.Variable(xavier_init([h_dim, z_dim]))
Q_b2_sigma = tf.Variable(tf.zeros(shape=[z_dim]))


def Q(X, c):
    inputs = tf.concat(axis=1, values=[X, c])
    h = tf.nn.relu(tf.matmul(inputs, Q_W1) + Q_b1)
    z_mu = tf.matmul(h, Q_W2_mu) + Q_b2_mu
    z_logvar = tf.matmul(h, Q_W2_sigma) + Q_b2_sigma
    return z_mu, z_logvar


def sample_z(mu, log_var):
    eps = tf.random.normal(shape=tf.shape(mu))
    return mu + tf.exp(log_var / 2) * eps

P_W1 = tf.Variable(xavier_init([z_dim + y_dim, h_dim]))
P_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

P_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
P_b2 = tf.Variable(tf.zeros(shape=[X_dim]))


def P(z, c):
    inputs = tf.concat(axis=1, values=[z, c])
    h = tf.nn.relu(tf.matmul(inputs, P_W1) + P_b1)
    logits = tf.matmul(h, P_W2) + P_b2
    prob = tf.nn.sigmoid(logits)
    return prob, logits

z_mu, z_logvar = Q(X, c)
z_sample = sample_z(z_mu, z_logvar)
_, logits = P(z_sample, c)

X_samples, _ = P(z, c)

recon_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=X), 1)
kl_loss = 0.5 * tf.reduce_sum(tf.exp(z_logvar) + z_mu**2 - 1. - z_logvar, 1)
vae_loss = tf.reduce_mean(recon_loss + kl_loss)



Instructions for updating:
non-resource variables are not supported in the long term


In [10]:
### generating sample outputs after training
def generate_sample():
    samples = []
    gen_labels =[]
    for r in range(10):
        for index in range(y_dim):
            gen_labels = gen_labels + [index]*mb_size
            y = np.zeros([mb_size, y_dim])
            y[range(mb_size), index] = 1
            samples.extend(sess.run(X_samples,
                                   feed_dict={z: np.random.randn(mb_size, z_dim), c: y}))

    gen_samples = np.array(samples).round(decimals=2)
    gen_labels = np.array(gen_labels)
    print(gen_samples.shape)
    print(gen_labels.shape)
    
    return gen_samples, gen_labels

# MLP Model

In [11]:
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input, Flatten
from tensorflow.keras.models import Sequential


def build_model(input_shape=(64,), num_classes=10):
    """

    :param input_shape: shape of input_data
    :param num_classes: number of classes
    :return: keras.model.sequential compiled with categorical cross-entropy loss
    """
    model = Sequential([
        Input(shape=input_shape),
        Dense(32, activation="relu"),
        BatchNormalization(),
        Dense(64, activation="relu"),
        BatchNormalization(),
        Flatten(),
        Dropout(0.5),
        Dense(num_classes, activation="softmax"),
    ])

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    model.summary()
    return model

In [12]:
def post_process_results(b_list, filename='default.csv'):
    total_df = pd.DataFrame(b_list[0]).transpose()
    print('number of runs: {}'.format(len(b_list)))
    for r_dict in b_list[1:]:
        temp = pd.DataFrame(r_dict).transpose()
        total_df = total_df.add(temp)
        
    average_pd = total_df/10.0
    average_pd.to_csv(filename, sep=',')
    
    return average_pd

# Baseline Experiment

In [13]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

baseline_list =[]
for i in range(10):
    
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=40)
    y_train = LabelEncoder().fit_transform(y_train)
    y_test = LabelEncoder().fit_transform(y_test)

    baseline_model = build_model((7,), 5)
    batch_size=8
    epochs=2
    y_train_oh = np.array(tf.keras.utils.to_categorical(y_train, num_classes=5, dtype='float32'))
    test_y = np.array(tf.keras.utils.to_categorical(y_test, num_classes=5, dtype='float32'))
    history_baseline = baseline_model.fit(X_train, y_train_oh, batch_size=batch_size, 
                        epochs=epochs, validation_data=(X_test, test_y))
    score_baseline = baseline_model.evaluate(X_test, test_y, verbose=0)
    print('baseline test loss: ',score_baseline[0])
    print('baseline test accuracy: ', score_baseline[1] )

    y_pred_baseline_oh = baseline_model.predict(X_test)
    y_pred_baseline = y_pred_baseline_oh.argmax(axis=-1)
    baseline_list.append(classification_report(y_test, y_pred_baseline, output_dict=True))

Instructions for updating:
Colocations handled automatically by placer.


/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                256       
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 3

2021-11-02 11:00:12.809986: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-02 11:00:12.820387: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2021-11-02 11:00:12.821732: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-11-02 11:00:12.822757: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library

261/261 [==============================] - 0s 993us/sample - loss: 2.0954 - acc: 0.3103 - val_loss: 1.5338 - val_acc: 0.5455
Epoch 2/2
261/261 [==============================] - 0s 215us/sample - loss: 1.3002 - acc: 0.5824 - val_loss: 1.5168 - val_acc: 0.3939
baseline test loss:  1.5168015306646174
baseline test accuracy:  0.3939394


...

261/261 [==============================] - 0s 2ms/sample - loss: 1.7491 - acc: 0.3716 - val_loss: 1.5247 - val_acc: 0.3636
Epoch 2/2
261/261 [==============================] - 0s 218us/sample - loss: 1.0153 - acc: 0.5824 - val_loss: 1.4367 - val_acc: 0.6212
baseline test loss:  1.4367356083609841
baseline test accuracy:  0.6212121


/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisi

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 32)                256       
_________________________________________________________________
batch_normalization_18 (Batc (None, 32)                128       
_________________________________________________________________
dense_28 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_19 (Batc (None, 64)                256       
_________________________________________________________________
flatten_9 (Flatten)          (None, 64)                0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 5)                

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


261/261 [==============================] - 0s 2ms/sample - loss: 1.8049 - acc: 0.4789 - val_loss: 1.5219 - val_acc: 0.5152
Epoch 2/2
261/261 [==============================] - 0s 225us/sample - loss: 1.2513 - acc: 0.5747 - val_loss: 1.4557 - val_acc: 0.7121
baseline test loss:  1.4557068781419233
baseline test accuracy:  0.7121212


In [14]:
post_process_results(baseline_list, 'results_csv/ecoli_baseline_cnn.csv')

number of runs: 10


,precision,recall,f1-score,support
0,0.805897,0.553571,0.599453,28.000000
1,0.415236,0.741176,0.478777,17.000000
2,0.185079,0.222222,0.140932,9.000000
3,0.250000,0.200000,0.196667,3.000000
4,0.409596,0.322222,0.294245,9.000000
accuracy,0.509091,0.509091,0.509091,0.509091
macro avg,0.413161,0.407838,0.342015,66.000000
weighted avg,0.541306,0.509091,0.445916,66.000000


# Random UnderSampling

In [22]:
undersampling_list =[]    
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
    ros = RandomUnderSampler(random_state=42)
    X_train, y_train = ros.fit_resample(X_train, y_train)
    y_train = LabelEncoder().fit_transform(y_train)
    y_test = LabelEncoder().fit_transform(y_test)

    baseline_model =  build_model((7,), 5)
    batch_size=8
    epochs=2
    y_train_oh = np.array(tf.keras.utils.to_categorical(y_train, num_classes=y_dim, dtype='float32'))
    test_y = np.array(tf.keras.utils.to_categorical(y_test, num_classes=y_dim, dtype='float32'))
    
    _ = baseline_model.fit(X_train, y_train_oh, batch_size=batch_size, 
                        epochs=epochs, validation_data=(X_test, test_y))
    
    score_baseline = baseline_model.evaluate(X_test, test_y, verbose=0)
    print('undersampling test loss: ',score_baseline[0])
    print('undersampling test accuracy: ', score_baseline[1] )

    y_pred_baseline_oh = baseline_model.predict(X_test)
    y_pred_baseline = y_pred_baseline_oh.argmax(axis=-1)
    undersampling_list.append(classification_report(y_test, y_pred_baseline, output_dict=True))

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Model: "sequential_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_123 (Dense)            (None, 32)                256       
_________________________________________________________________
batch_normalization_82 (Batc (None, 32)                128       
_________________________________________________________________
dense_124 (Dense)            (None, 64)                2112      
_________________________________________________________________
batch_normalization_83 (Batc (None, 64)                256       
_________________________________________________________________
flatten_41 (Flatten)         (None, 64)                0         
_________________________________________________________________
dropout_41 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_125 (Dense)            (None, 5)               

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


85/85 [==============================] - 2s 20ms/sample - loss: 2.2691 - acc: 0.2000 - val_loss: 1.5716 - val_acc: 0.3939
Epoch 2/2
85/85 [==============================] - 0s 749us/sample - loss: 1.4346 - acc: 0.3647 - val_loss: 1.5626 - val_acc: 0.2727
undersampling test loss:  1.5625562812342788
undersampling test accuracy:  0.27272728


/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisi

...

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisi

Model: "sequential_50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_150 (Dense)            (None, 32)                256       
_________________________________________________________________
batch_normalization_100 (Bat (None, 32)                128       
_________________________________________________________________
dense_151 (Dense)            (None, 64)                2112      
_________________________________________________________________
batch_normalization_101 (Bat (None, 64)                256       
_________________________________________________________________
flatten_50 (Flatten)         (None, 64)                0         
_________________________________________________________________
dropout_50 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_152 (Dense)            (None, 5)               

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


85/85 [==============================] - 2s 25ms/sample - loss: 2.6859 - acc: 0.2353 - val_loss: 1.6146 - val_acc: 0.0909
Epoch 2/2
85/85 [==============================] - 0s 779us/sample - loss: 1.5529 - acc: 0.4471 - val_loss: 1.6205 - val_acc: 0.1061
undersampling test loss:  1.620524738774155
undersampling test accuracy:  0.10606061


/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisi

In [23]:
post_process_results(undersampling_list, 'results_csv/ecoli_undersamling.csv')

number of runs: 10


,precision,recall,f1-score,support
0,0.291608,0.206667,0.138561,30.000000
1,0.241071,0.286667,0.160547,15.000000
2,0.086357,0.433333,0.140614,6.000000
3,0.060882,0.266667,0.087198,3.000000
4,0.084091,0.058333,0.051786,12.000000
accuracy,0.221212,0.221212,0.221212,0.221212
macro avg,0.152802,0.250333,0.115741,66.000000
weighted avg,0.213245,0.221212,0.125633,66.000000


# Random OverSampling

In [24]:
oversampling_list =[]    
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
    ros = RandomOverSampler(random_state=42)
    X_train, y_train = ros.fit_resample(X_train, y_train)
    y_train = LabelEncoder().fit_transform(y_train)
    y_test = LabelEncoder().fit_transform(y_test)

    baseline_model =  build_model((7,), 5)
    batch_size=8
    epochs=2
    y_train_oh = np.array(tf.keras.utils.to_categorical(y_train, num_classes=y_dim, dtype='float32'))
    test_y = np.array(tf.keras.utils.to_categorical(y_test, num_classes=y_dim, dtype='float32'))
    
    _ = baseline_model.fit(X_train, y_train_oh, batch_size=batch_size, 
                        epochs=epochs, validation_data=(X_test, test_y))
    
    score_baseline = baseline_model.evaluate(X_test, test_y, verbose=0)
    print('undersampling test loss: ',score_baseline[0])
    print('undersampling test accuracy: ', score_baseline[1] )

    y_pred_baseline_oh = baseline_model.predict(X_test)
    y_pred_baseline = y_pred_baseline_oh.argmax(axis=-1)
    oversampling_list.append(classification_report(y_test, y_pred_baseline, output_dict=True))

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Model: "sequential_51"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_153 (Dense)            (None, 32)                256       
_________________________________________________________________
batch_normalization_102 (Bat (None, 32)                128       
_________________________________________________________________
dense_154 (Dense)            (None, 64)                2112      
_________________________________________________________________
batch_normalization_103 (Bat (None, 64)                256       
_________________________________________________________________
flatten_51 (Flatten)         (None, 64)                0         
_________________________________________________________________
dropout_51 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_155 (Dense)            (None, 5)               

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


565/565 [==============================] - 2s 4ms/sample - loss: 1.5931 - acc: 0.4248 - val_loss: 1.6076 - val_acc: 0.1061
Epoch 2/2
565/565 [==============================] - 0s 506us/sample - loss: 1.0124 - acc: 0.6301 - val_loss: 1.4849 - val_acc: 0.3788
undersampling test loss:  1.484929250948357
undersampling test accuracy:  0.37878788


/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisi

Model: "sequential_60"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_180 (Dense)            (None, 32)                256       
_________________________________________________________________
batch_normalization_120 (Bat (None, 32)                128       
_________________________________________________________________
dense_181 (Dense)            (None, 64)                2112      
_________________________________________________________________
batch_normalization_121 (Bat (None, 64)                256       
_________________________________________________________________
flatten_60 (Flatten)         (None, 64)                0         
_________________________________________________________________
dropout_60 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_182 (Dense)            (None, 5)               

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


565/565 [==============================] - 3s 5ms/sample - loss: 1.3533 - acc: 0.5044 - val_loss: 1.6925 - val_acc: 0.2121
Epoch 2/2
565/565 [==============================] - 0s 533us/sample - loss: 0.9429 - acc: 0.6637 - val_loss: 1.6120 - val_acc: 0.2424
undersampling test loss:  1.612029133421002
undersampling test accuracy:  0.24242425


/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisi

In [25]:
post_process_results(undersampling_list, 'results_csv/ecoli_oversamling.csv')

number of runs: 10


,precision,recall,f1-score,support
0,0.291608,0.206667,0.138561,30.000000
1,0.241071,0.286667,0.160547,15.000000
2,0.086357,0.433333,0.140614,6.000000
3,0.060882,0.266667,0.087198,3.000000
4,0.084091,0.058333,0.051786,12.000000
accuracy,0.221212,0.221212,0.221212,0.221212
macro avg,0.152802,0.250333,0.115741,66.000000
weighted avg,0.213245,0.221212,0.125633,66.000000


# Augmentation Experiment

In [20]:
augment_list =[]
for i in range(10):
    
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=40)
    y_train = LabelEncoder().fit_transform(y_train)
    y_test = LabelEncoder().fit_transform(y_test)
    y_train_oh = np.array(tf.keras.utils.to_categorical(y_train, num_classes=5, dtype='float32'))
    test_y = np.array(tf.keras.utils.to_categorical(y_test, num_classes=5, dtype='float32'))
    
    
    
    solver = tf.compat.v1.train.AdamOptimizer().minimize(vae_loss)
    sess = tf.compat.v1.Session ()
    sess.run(
    tf.compat.v1.global_variables_initializer())

    i = 0

    for it in tqdm(range(50000)):
        ind = np.random.choice(X_train.shape[0], mb_size)
        X_mb = np.array(X_train[ind])
        y_mb = np.array(y_train_oh[ind])

        _, loss = sess.run([solver, vae_loss], feed_dict={X: X_mb, c: y_mb})

    
    gen_samples, gen_labels = generate_sample()
    x = np.concatenate([X_train, gen_samples])
    y = np.concatenate([y_train, gen_labels])
    x = np.array(x)
    y_oh = np.array(tf.keras.utils.to_categorical(y, num_classes=5, dtype='float32'))
    
    aug_model = build_model((7,), 5)
    batch_size=8
    epochs=2
    hist = aug_model.fit(x, y_oh, batch_size=batch_size, epochs=epochs, 
                            validation_data=(X_test, test_y))
    y_pred_aug_oh = aug_model.predict(X_test)
    y_pred_aug = y_pred_aug_oh.argmax(axis=-1)
    augment_list.append(classification_report(y_test, y_pred_aug, output_dict=True) )
    

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:42<00:00, 1175.70it/s]


(800, 7)
(800,)
Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_93 (Dense)             (None, 32)                256       
_________________________________________________________________
batch_normalization_62 (Batc (None, 32)                128       
_________________________________________________________________
dense_94 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_63 (Batc (None, 64)                256       
_________________________________________________________________
flatten_31 (Flatten)         (None, 64)                0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_95 (Dense)             (None, 5

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


1061/1061 [==============================] - 1s 1ms/sample - loss: 0.9144 - acc: 0.6484 - val_loss: 1.5479 - val_acc: 0.5152
Epoch 2/2
1061/1061 [==============================] - 0s 327us/sample - loss: 0.5680 - acc: 0.8030 - val_loss: 1.0544 - val_acc: 0.6515


/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:40<00:00, 1241.28it/s]


...

1061/1061 [==============================] - 2s 2ms/sample - loss: 1.0249 - acc: 0.6635 - val_loss: 1.5877 - val_acc: 0.4545
Epoch 2/2
1061/1061 [==============================] - 0s 370us/sample - loss: 0.5356 - acc: 0.8341 - val_loss: 1.1744 - val_acc: 0.6061


/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:41<00:00, 1213.57it/s]


(800, 7)
(800,)
Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_120 (Dense)            (None, 32)                256       
_________________________________________________________________
batch_normalization_80 (Batc (None, 32)                128       
_________________________________________________________________
dense_121 (Dense)            (None, 64)                2112      
_________________________________________________________________
batch_normalization_81 (Batc (None, 64)                256       
_________________________________________________________________
flatten_40 (Flatten)         (None, 64)                0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_122 (Dense)            (None, 5

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


1061/1061 [==============================] - 2s 2ms/sample - loss: 1.5339 - acc: 0.4722 - val_loss: 1.4255 - val_acc: 0.3333
Epoch 2/2
1061/1061 [==============================] - 0s 418us/sample - loss: 0.8523 - acc: 0.6418 - val_loss: 0.9224 - val_acc: 0.7576


In [21]:
post_process_results(augment_list, 'results_csv/ecoli_VAE.csv')

number of runs: 10


,precision,recall,f1-score,support
0,0.920077,0.835714,0.850790,28.000000
1,0.704534,0.623529,0.636940,17.000000
2,0.638557,0.711111,0.613017,9.000000
3,0.535833,0.733333,0.555556,3.000000
4,0.766572,0.655556,0.637703,9.000000
accuracy,0.734848,0.734848,0.734848,0.734848
macro avg,0.713115,0.711849,0.658801,66.000000
weighted avg,0.787771,0.734848,0.720807,66.000000
